![Header](img/header_1.jpg)

In [ ]:
%matplotlib widget
import sofa
HRIR_path = "hrir/ITA_Artificial_Head_5x5_44100Hz.sofa"
HRIR_dataset = sofa.Database.open(HRIR_path)
import helper_functions as hf

from IPython.display import Audio
from scipy import signal 
from ipywidgets import Button, IntSlider, Output, Layout
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits.mplot3d.axes3d import Axes3D

# extract the respective positions from the HRIR dataset:
source_positions = HRIR_dataset.Source.Position.get_values(system="cartesian")
listener_position = np.squeeze(HRIR_dataset.Listener.Position.get_values(system="cartesian"))
listener_up = np.squeeze(HRIR_dataset.Listener.Up.get_values(system="cartesian"))
listener_view = np.squeeze(HRIR_dataset.Listener.View.get_values(system="cartesian"))

# Binauralsynthese

Das menschliche Gehör kann Entfernung und Richtung, also Position, einer Quelle bestimmen. Durch 
Beugungen und Reflexionen am Körper wird das von der Quelle kommende  Signal verändert und 
trifft in unterschiedlicher Form auf beiden Ohren. Aus der Art der Veränderung und dem Unterschied 
zwischen den beiden Ohren, wird in unserem auditiven Cortex eine Richtung bestimmt. Ist das 
Übertragungsverhalten von der Schallquelle zu beiden Ohren bekannt, kann man ein Signal im 
Vorhinein schon so bearbeiten (filtern), dass es einen Richtungseindruck erzeugt, der es ermöglicht 
virtuell platzierte Schallquellen im Raum zu platzieren. Dem Hörer kann man also eine beliebige 
Position der Quelle vortäuschen. Dieses Verfahren wird Binauralesynthese genannt,
die auf der Widergabe des binauralen Signals für das linke und rechte Ohr beruht.

# 1. Kopfbezogene Außenohrübertragungsfunktionen (HRTFs)

Die kopfbezogene Außenohrübertragungsfunktionen beschreibt das bereits erwähnte Übertragungsverhalten von einer Schallquelle, die sich an einer beliebigen Position im Raum befinden kann, zu den Ohren. Sie beinhaltet alle Informationen
darüber, wie eine Schallwelle, die von einer Quelle aus einer bestimmten Richtung ausgestrahlt wird, vom 
Körper der Zuhörenden beeinflusst wird, bevor sie das Trommelfell erreicht. Die Welle wird hier an
den Schultern, dem Kopf und der Ohrmuschel gebeugt und reflektiert.
Für die Binauralsynthese wird angenommen, dass die HRTF sich als eine Funktion beschreiben lässt,
die von der Position der Quelle relativ zum linken und rechten Ohr abhängt. In der Praxis wird die HRTF
durch Messungen in konstanten Winkelschritten um den Kopf mit konstantem Abstand ermittelt. Für die Binauralsynthese
wird dann die HRTF der Richtung, die sich am nächsten an der angestrebten Quellposition befindet, ausgewählt.

Die Richtung der Schallquelle
ist beschrieben durch den Elevationswinkel $\vartheta$, und den Azimuthwinkel $\varphi$, wobei $\vartheta$ die Rotation um die interaurale Achse ($y$-Achse), die beide Ohren verbindet, beschreibt und $\varphi$ die Rotation um die vertikale Achse ($z$-Achse) beschreibt, die 
von oben durch den Kopf verläuft. Ein positiver Elevationswinkel beschreibt eine Quellposition von oben, ein negativer Winkel eine Quellposition von unten. Ein Azimuthwinkel von $+90^\circ$ entspricht einer Quellposition links, ein Azimuthwinkel bei $-90^\circ$ einer Quellposition rechts. Für $0^\circ$ befindet sich die Quelle vorne, was der $x$-Achse entspricht.

<img src="img/pti_binaural_synthesis_xyz.png" width="800" height="400">



Um die Position einer Schallquelle in einer Umgebung oder einem Raum zu beschreiben werden 
allgemein Kartesische Koordinaten $(x,y,z)$ verwendet. Es wird hier angenommen, dass der Kopf sich im 
Ursprung des Koordinatensystems befindet. Die folgende Grafik visualisiert die Position der Quelle relativ 
zum Kopf. Die Grafik lässt sich zur besseren Erkennung per Maus rotieren.

In [ ]:
## Koordinaten hier anpassen
x = 1
y = 1
z = 0

source_position = np.array([x, y, z])


fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(*source_position/np.linalg.norm(source_position), s=30, 
           label='Source (x={}, y={}, z={})'.format(x, y, z))
ax.quiver(*listener_position, *listener_view*0.9, color='C3', label='View vector (x-axis)')
ax.quiver(*listener_position, *source_position/np.linalg.norm(source_position)*0.9, color='C0')
ax.quiver(*listener_position, *listener_up, color='C2', label='Up vector (z-axis)')
ax.quiver(0, 0, 0, 0, 0.9, 0, color='C1', label='y-axis')
ax.scatter(*listener_position, s=150, label='Head', color='k')
ax.set_xlabel('x in [m]')
ax.set_ylabel('y in [m]')
ax.set_zlabel('z in [m]')
ax.set_xlim([-1, 1])
ax.set_ylim([-1, 1])
ax.set_zlim([-1, 1])
ax.set_xticks([-1, 0, 1])
ax.set_yticks([-1, 0, 1])
ax.set_zticks([-1, 0, 1])
ax.legend(ncol=3);

## Aufgabe 1: Berechnung des Elevations und Azimuthwinkels

Um die HRTF für die Richtung der Schallquelle zu ermitteln benötigt man nun den Elevations- und Azimuthwinkel.
Diese lassen sich berechnen als

$$
    r = \sqrt{x^2 + y^2 + z^2} \\
    \vartheta = \arcsin \left( \frac{z}{r} \right) \\
    \varphi = \arctan \left( \frac{y}{x} \right) \\ 
$$

**Aufgabe**: Berechnen Sie mit den gegebenen Formeln den Elevations- und Azimuthwinkel für die gegebenen $(x, y, z)$-Koordinaten. Verwenden Sie verschiedene Koordinatenpunkte um ihr Ergebnis zu verifizieren.

_Hinweis_:
Die benötigten Funktionen und speziellen Operationen sind

$\sqrt{a}$ : `np.sqrt(a)`

$a^2$ : `a**2`

$\arctan\left({a/b}\right)$ : `np.arctan(a, b)`

$\arcsin\left({a/b}\right)$ : `np.arcsin(a/b)`


In [ ]:
## Koordinaten hier anpassen
x = 0 
y = 1
z = 0

## Lösung hier eintragen
r = 1
azimuth = 0
elevation = 0
## Lösung Ende

print(f'Azimuth = {azimuth}')
print(f'Elevation = {elevation}')

---  
# 2. Die Interaurale Zeitdifferenz und die Interaurale Level-Differenz

Die beiden Hauptmechanismen, die vom menschlichen Gehirn verwendet werden, um eine Schallquelle zu lokalisieren,
basieren auf der _Interauralen Leveldifferenz (ILD)_  und der _Interauralen Zeitdifferenz (ITD)_. Die ITD beschreibt die
zeitliche Verzögerung eines Schallereignisses, der zwischen den beiden Ohren als Resultat der Schallgeschwindigkeit ($c = 343$ m/s) auftritt. Die ILD beschreibt den Lautsstärkeunterschied zwischen beiden Ohren, der durch die Abschattung des durch den Kopf von einem Schallerergbis abgewandten Ohres entsteht. Bei tiefen Frequenzen wertet das menschliche Gehirn vor allem die Zeitdifferenz aus, wobei bei höheren Frequenzen vor allem die Leveldifferenz ausschlaggebend für die Lokalisation ist.

## Aufgabe 2.1: Berechnung der Interauralen Zeitdifferenz

Schallwellen breiten sich mit einer Geschwindigkeit von ungefähr $c = 343$ m/s aus. Berechnen Sie die Interaurale Zeitdifferenz für eine Schallquelle, die genau von links auf einen Kopf mit einem Durchmesser von $d = 15$ cm trifft. Nehmen Sie an, dass die Schallquelle nicht durch den Kopf wandern kann, sondern auf einer Kreisbahn mit dem Durchmesser des Kopfes um diesen herum wandert.

In [ ]:
c = 343 # Schallgeschwindigkeit in m/s
d = 15e-2 # Durchmesser in m

pi = np.pi

# Lösung hier eintragen
# Interarale Zeitdifferenz in s
ITD = 0 
print(f"ITD = {ITD*1e3} ms")

---

In [ ]:
sampling_rate = HRIR_dataset.Data.SamplingRate.get_values(indices={"M":0})
from scipy import signal 
import ipywidgets

def get_ITD(HRIR, sampling_rate=44100):
    """
    Get the interaural time difference (ITD) for a specified HRIR.

    Parameters
    ----------
    HRIR : numpy.ndarray
        The HRIR for a single direction.
    sampling_rate : integer
        The sampling rate of the HRIR.

    Returns
    -------
    ITD : double
        The interaural time difference (ILD).
    """
    sos = signal.butter(10, 1.5e3, btype='low', output='sos', fs=sampling_rate)
    HRIR = signal.sosfilt(sos, HRIR)
    n_samples = HRIR.shape[-1]
    t = np.arange(0, n_samples)/sampling_rate
    corr = signal.correlate(HRIR[1], HRIR[0])
    corr_lags = np.arange(-n_samples + 1, n_samples)/sampling_rate
    
    ITD = np.abs(corr_lags[np.argmax(np.abs(corr))])
    
    return ITD

def get_ILD(HRIR):
    """
    Get the interaural level difference (ILD) for a specified HRIR.

    Parameters
    ----------
    HRIR : numpy.ndarray
        The HRIR for a single direction.

    Returns
    -------
    ILD : double
        The interaural level difference (ILD).
    """
    energy = np.sum(np.abs(HRIR)**2, axis=-1)
    ILD = 10*np.log10(energy[0]/energy[1])
    
    return ILD


## Aufgabe 2.2: Ermittlung der ITA aus der HRTF

Die Interaurale Zeit- und Leveldifferenz lassen sich wie eingangs beschrieben aus der HRTF bestimmen. Die Interaurale Zeitdiffernz lässt sich hier als die Zeitdifferenz zwischen den Maxima für das linke und das rechte Ohr im Zeitbereich bestimmen (siehe Plot unten links).
Die Interaurale Leveldifferenz lässt sich als die Fläche zwischen den Funktionen für das linke und das rechte Ohr berechnen (siehe Plot unten rechts)
Verwenden Sie die Slider für den Azimuthwinkel um ihr Ergebnis aus der vorherigen Aufgabe zu überprüfen.
## Aufgabe 2.3 Analyse der Interauralen Parameter 
Verwenden Sie beide Slider um Positionen zu Ermitteln an denen die ILD und die ITD 0 werden. Überlegen Sie, wie dies zu Stande kommt. Welche Auswirkungen hat dies auf die Ortungsfähigkeit des Menschen. Geben Sie eine Deutung.

In [ ]:
slider_azimuth = ipywidgets.IntSlider(
    value=-35, min=-90, max=90, step=5, description='Azimuth', continuous_update=False)
slider_elevation = ipywidgets.IntSlider(
    value=0, min=-90, max=90, step=5, description='Elevation', continuous_update=False)

interactive_panel = ipywidgets.interact(
    hf.plot_HRIR_at_direction,
    HRIR_dataset=ipywidgets.fixed(HRIR_dataset),
    ILD_function = ipywidgets.fixed(get_ILD),
    ITD_function = ipywidgets.fixed(get_ITD),
    azimuth=slider_azimuth,
    elevation=slider_elevation)


---
# 3. Erstellen der Binauralsynthese

Der letzte Aufgabenteil beschäftigt sich mit dem Erstellen der Binauralsynthese. Hierfür wird ein monoaurales Signal $s(t)$, das keinerlei Richtungsinformationen enthält mit der HRTF für die gewünschte Richtung gefaltet. Die Faltung lässt sich mathematisch als 

$$
y_{l}(t, \vartheta, \varphi) = \sum s(t) h_l(t - \tau, \vartheta, \varphi)\\
y_{r}(t, \vartheta, \varphi) = \sum s(t) h_r(t - \tau, \vartheta, \varphi)
$$

schreiben, wobei $h_l$ und $h_r$ die HRTF für das linke und rechte Ohr beschreiben, und $y_l$ und $y_r$ das resultierende binaurale Signal für das jeweilige Ohr sind. Es sind also nur die HRTF und das resultierende binaurale Signal verschieden für beide Ohren. Das monoaurale Quellsignal ist unabhängig vom Ohr und nur abhängig von der Schallquelle. Die Faltung und Binauralsynthese sind im Folgenden bereit implementiert und werden nur verwendet.

## Aufgabe 3.1: Vergleich von monoauraler und binauraler Wiedergabe
**Die folgenden Hörbeispiele erfordern zwingend die Verwendung von Kopfhörern**

Im folgenden Widget hören Sie ein Audiosignal, das entweder monoaural, also das selbe Signal auf beiden Ohren, oder binaural wiedergegeben wird. Die Wiedergabemethode lässt sich über die _binaural_ Checkbox umschalten. Vergleichen Sie beide Methoden, verschieben Sie hierfür auch die Quelle mittels des Azimuth sliders. Wie hört sich das monoaurale Signal im Vergleich zum binauralen Signal an?

In [ ]:
from IPython.display import Audio

def binaural_synthesis_evaluation(instrument='guitar', binaural=False, azi=90, ele=0):
    source_signal = hf.read_wav('audio/' + instrument + '.wav')
    hrir = hf.get_HRIR_at_direction(HRIR_dataset, azi, ele)
    
    if not binaural:
        out = np.vstack((source_signal, source_signal))
    else:
        out = np.vstack((
            signal.oaconvolve(hrir[1, 80:80+128], source_signal),
            signal.oaconvolve(hrir[0, 80:80+128], source_signal)))
    return Audio(data=out, rate=44100, autoplay=True)

instruments = ['guitar', 'horns', 'vocals']

instrument_selector = ipywidgets.Dropdown(
    options=instruments,
    value='guitar',
    description="Instrument"
)

slider_azimuth = ipywidgets.IntSlider(
    value=0, min=-180, max=180, step=5, 
    description='Azimuth', continuous_update=False,
    layout=Layout(height='auto', width='auto'))
slider_elevation = ipywidgets.IntSlider(
    value=0, min=-90, max=90, step=5,
    description='Elevation', continuous_update=False,
    layout=Layout(height='auto', width='auto'))

ipywidgets.interact(
    binaural_synthesis_evaluation,
    instrument=instrument_selector,
    azi=slider_azimuth,
    ele=slider_elevation,
)

## Aufgabe 3.2: Binauralsynthese mit mehreren Quellen
**Die folgenden Hörbeispiele erfordern zwingend die Verwendung von Kopfhörern**

Im folgenden Widget können Sie zwei verschiedene Audiosignale binaural wiedergeben. Evaluieren Sie die Binauralsynthese hinsichtlich der Plausibilität der Wiedergabe. Verschieben Sie auch hier wieder auch die Quellen mittels des Azimuth- und Elevationsslider. Es kann hilfreich sein eine Quelle an einer beliebig gewählten Position zu lassen und die andere Position zu variieren. Sie können die Lautstärke der Quellen über den Gain Regler anpassen. Beantworten Sie zur Evaluation die folgenden Fragen:
1. Klingt die hier erstellte Binauralsynthese realistisch?
2. Welcher Effekt tritt auf, wenn Sie die Schallquelle an Positionen (oder in deren Nähe) bewegen, an denen die ITD und ILD Null werden?

In [ ]:
def play_binaural_signals(
        horns_gain, horns_azi, horns_ele, horns_sig,
        git_gain, git_azi, git_ele, git_sig, out,
    ):
    hrir_guitar = hf.get_HRIR_at_direction(HRIR_dataset, git_azi, git_ele)
    hrir_horns = hf.get_HRIR_at_direction(HRIR_dataset, horns_azi, horns_ele)
    binaural_guitar = np.vstack((
        signal.oaconvolve(hrir_guitar[1, 80:80+128], audio_data_guitar),
        signal.oaconvolve(hrir_guitar[0, 80:80+128], audio_data_guitar)))

    binaural_horns = np.vstack((
        signal.oaconvolve(hrir_horns[1, 80:80+128], audio_data_horns),
        signal.oaconvolve(hrir_horns[0, 80:80+128], audio_data_horns)))

    binaural_mix = 10**(horns_gain/20) * binaural_horns + 10**(git_gain/20) * binaural_guitar
    if out is None:
        return Audio(data=binaural_mix, rate=44100, autoplay=True)
    else:
        with out:
            audio_out = Audio(data=binaural_mix, rate=44100, autoplay=True, loop=True)

In [ ]:
from ipywidgets import GridspecLayout
from ipywidgets import Button, Layout, jslink, IntText, IntSlider, Output, HBox

audio_data_guitar = hf.read_wav('audio/guitar.wav')
audio_data_horns = hf.read_wav('audio/vocals.wav')

def create_expanded_button(description, button_style):
    return Button(
        description=description,
        button_style=button_style,
        layout=Layout(height='auto', width='auto'))

slider_azimuth_git = ipywidgets.IntSlider(
    value=90, min=-180, max=180, step=5, 
    description='Azimuth [deg]', continuous_update=False,
    layout=Layout(height='auto', width='auto'))
slider_elevation_git = ipywidgets.IntSlider(
    value=0, min=-90, max=90, step=5,
    description='Elevation [deg]', continuous_update=False,
    layout=Layout(height='auto', width='auto'))
slider_gain_git = ipywidgets.IntSlider(
    value=0, min=-50, max=0, step=1,
    description='Gain [dB]', continuous_update=False,
    layout=Layout(height='auto', width='auto'))


slider_azimuth_horns = ipywidgets.IntSlider(
    value=0, min=-180, max=180, step=5, 
    description='Azimuth [deg]', continuous_update=False,
    layout=Layout(height='auto', width='auto'))
slider_elevation_horns = ipywidgets.IntSlider(
    value=0, min=-90, max=90, step=5,
    description='Elevation [deg]', continuous_update=False,
    layout=Layout(height='auto', width='auto'))
slider_gain_horns = ipywidgets.IntSlider(
    value=0, min=-50, max=0, step=1,
    description='Gain [dB]', continuous_update=False,
    layout=Layout(height='auto', width='auto'))


grid = GridspecLayout(5, 2, height='200px')
grid[0, 0] = create_expanded_button('Source 1', 'success')
grid[1, 0] = slider_azimuth_horns
grid[2, 0] = slider_elevation_horns
grid[3, 0] = slider_gain_horns
grid[0, 1] = create_expanded_button('Source 2', 'success')
grid[1, 1] = slider_azimuth_git
grid[2, 1] = slider_elevation_git
grid[3, 1] = slider_gain_git

panel = ipywidgets.interact(
    play_binaural_signals,
    horns_gain=slider_gain_horns,
    horns_azi=slider_azimuth_horns,
    horns_ele=slider_elevation_horns,
    horns_sig=ipywidgets.fixed(audio_data_horns),
    git_gain=slider_gain_git,
    git_azi=slider_azimuth_git,
    git_ele=slider_elevation_git,
    git_sig=ipywidgets.fixed(audio_data_guitar),
    out=ipywidgets.fixed(None))

grid

*Note: All audio files have been engineered and recorded by TELEFUNKEN Elektroakustik and are presented for educational and demonstrational purposes only.*